In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# 学習データ、テストデータの読み込み
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
# 名前、客室番号、チケット番号のカラムは除外
df_train = df_train.drop('Name', axis=1)
df_train = df_train.drop('Ticket', axis=1)
df_train = df_train.drop('Cabin', axis=1)
df_test = df_test.drop('Name', axis=1)
df_test = df_test.drop('Ticket', axis=1)
df_test = df_test.drop('Cabin', axis=1)
df_train.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [4]:
# 性別、乗船港のカラムを数値化
X_train = df_train.replace({'Sex': {'male': 0, 'female': 1}})
X_train = X_train.replace({'Embarked': {'S': 0, 'C': 1, 'Q': 2}})
df_test = df_test.replace({'Sex': {'male': 0, 'female': 1}})
df_test = df_test.replace({'Embarked': {'S': 0, 'C': 1, 'Q': 2}})
# 欠損値の除去
X_train = X_train.dropna()
df_test = df_test.dropna()
X_train.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,0.0
1,2,1,1,1,38.0,1,0,71.2833,1.0
2,3,1,3,1,26.0,0,0,7.9250,0.0
3,4,1,1,1,35.0,1,0,53.1000,0.0
4,5,0,3,0,35.0,0,0,8.0500,0.0


In [5]:
# 学習データについて、特徴量と正解ラベルを分離
Y_train = X_train['Survived']
X_train = X_train.drop('Survived', axis=1)
X_train.head(5)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,0,22.0,1,0,7.2500,0.0
1,2,1,1,38.0,1,0,71.2833,1.0
2,3,3,1,26.0,0,0,7.9250,0.0
3,4,1,1,35.0,1,0,53.1000,0.0
4,5,3,0,35.0,0,0,8.0500,0.0


In [6]:
# 線形SVMで学習
model = SVC(kernel='linear', random_state=None)
model.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [7]:
# 予測モデルの評価
# 学習データに対する精度
pred_train = model.predict(X_train)
accuracy_train = accuracy_score(Y_train, pred_train)
print('学習データに対する精度 = {}'.format(accuracy_train))

学習データに対する精度 = 0.7865168539325843


In [8]:
# テストデータに対しても予測
pred_test = model.predict(df_test)

In [9]:
# 予測結果の提出用にCSV出力
submission = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': pred_test}, columns=['PassengerId', 'Survived'])
np.savetxt('/kaggle/working/submission.csv', submission, delimiter=',', fmt='%d')